In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# NEW TOOLS FOR CLASSIFICATION
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

In [2]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# CHECK THE BALANCE
print(df['target'].value_counts())

target
1    357
0    212
Name: count, dtype: int64


In [30]:
cancer = load_breast_cancer()
# Basic dataset info
print("Dataset shape (patients, features):", X.shape)
print("\nTarget names:", cancer.target_names)

# Count class distribution
print("\nClass distribution:")
print(y.value_counts())

X.head()


Dataset shape (patients, features): (569, 30)

Target names: ['malignant' 'benign']

Class distribution:
target
1    357
0    212
Name: count, dtype: int64


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [31]:

# giving x inputs and y outputs
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = pd.Series(cancer.target, name="target")


# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Initialize scaler
scaler = StandardScaler()

# Fit ONLY on training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform test data using same scaler
X_test_scaled = scaler.transform(X_test)

# Optional sanity check
print("Mean of scaled training features (approx):", X_train_scaled.mean())
print("Std of scaled training features (approx):", X_train_scaled.std())

Mean of scaled training features (approx): -1.2493059090287476e-16
Std of scaled training features (approx): 1.0


In [22]:
# ==============================
# 5. LOGISTIC REGRESSION (RISK-AWARE)
# ==============================

log_reg = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',  # CRITICAL for medical data
    random_state=42
)

# Train model
log_reg.fit(X_train_scaled, y_train)


# ==============================
# 6. PREDICTION
# ==============================

y_pred = log_reg.predict(X_test_scaled)


# ==============================
# 7. EVALUATION (THE PART THAT MATTERS)
# ==============================

print("\n--- Accuracy ---")
print(accuracy_score(y_test, y_pred))

print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred))

print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))


--- Accuracy ---
0.956140350877193

--- Confusion Matrix ---
[[41  1]
 [ 4 68]]

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.91      0.98      0.94        42
           1       0.99      0.94      0.96        72

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114



In [7]:
# Get probabilities for the positive class (cancer)
y_prob = log_reg.predict_proba(X_test_scaled)[:, 0]

# Default threshold (0.5)
y_pred_default = (y_prob >= 0.5).astype(int)

# Lower threshold to reduce false negatives
y_pred_low_threshold = (y_prob >= 0.3).astype(int)

from sklearn.metrics import confusion_matrix

print("Confusion Matrix (threshold = 0.5)")
print(confusion_matrix(y_test, y_pred_default))

print("\nConfusion Matrix (threshold = 0.3)")
print(confusion_matrix(y_test, y_pred_low_threshold))

Confusion Matrix (threshold = 0.5)
[[ 1 41]
 [68  4]]

Confusion Matrix (threshold = 0.3)
[[ 1 41]
 [66  6]]
